In [4]:
import numpy as np
import pandas as pd
import tracktor as tr
import cv2
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

In [14]:
# offset to be added to x based on toxtrac arena selection
offset = 370 # fish - 0; flume - 245; spider - 75; termite - 370

# name of source video and paths
video = 'termite_video'
input_vidpath = '/mnt/ssd1/Documents/Vivek/tracktor/videos/' + video + '.mp4'
output_vidpath = '/mnt/ssd1/Documents/Vivek/tracktor/output/' + video + '_compared.mp4'
tracktor_datapath = '/mnt/ssd1/Documents/Vivek/tracktor/output/' + video + '_tracked.csv'
toxtrack_datapath = '/mnt/ssd1/Documents/Vivek/tracktor/output/' + video + '_toxtracked.txt'
codec = 'DIVX' # try other codecs if the default doesn't work ('DIVX', 'avc1', 'XVID') note: this list is non-exhaustive

In [15]:
## Open video
cap = cv2.VideoCapture(input_vidpath)
if cap.isOpened() == False:
    sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file')

## Open datafiles
df1 = pd.read_table(toxtrack_datapath, header=None, index_col=False, names=['frame', 'arena', 'id', 'pos_x', 'pos_y'])
df2 = pd.read_csv(tracktor_datapath)

## Video writer class to output video with contour and centroid of tracked object(s)
# make sure the frame size matches size of array 'final'
fourcc = cv2.VideoWriter_fourcc(*codec)
output_framesize = (int(cap.read()[1].shape[1]),int(cap.read()[1].shape[0]))
out = cv2.VideoWriter(filename = output_vidpath, fourcc = fourcc, fps = 30.0, frameSize = output_framesize, isColor = True)
last = 0

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    this = cap.get(1)
    if ret == True:
        frame = cv2.resize(frame, None, fx = 1.0, fy = 1.0, interpolation = cv2.INTER_LINEAR)
        
        if df1[df1['frame'] == this].empty == False:
            x1 = df1[df1['frame'] == this]['pos_x']+offset
            y1 = df1[df1['frame'] == this]['pos_y']
            for pts1 in range(0,len(x1)):
                cv2.circle(frame, (int(x1.values[pts1]),int(y1.values[pts1])), 5, (100,255,255), -1, cv2.LINE_AA)
        if df2[df2['frame'] == this].empty == False:
            x2 = df2[df2['frame'] == this]['pos_x']
            y2 = df2[df2['frame'] == this]['pos_y']
            for pts2 in range(0,len(x2)):
                cv2.circle(frame, (int(x2.values[pts2]),int(y2.values[pts2])), 5, (100,100,255), -1, cv2.LINE_AA)
        
        # Display the resulting frame
        out.write(frame)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) == 27:
            break
        
    if last == this:
        break
    
    last = this

# When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1